In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd
%cd /content/drive/My Drive/Colab Notebooks/NLP20211H/

/content
/content/drive/My Drive/Colab Notebooks/NLP20211H


#データ準備

In [3]:
!pip install mecab-python3
!pip install unidic-lite

     |████████████████████████████████| 491kB 9.6MB/s 
     |████████████████████████████████| 47.4MB 65kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-cp37-none-any.whl size=47658838 sha256=e786cf5ae09d82e0750c00fd58a3a7412f20c5319aaab3b9495fa3baf53ded2c
  Stored in directory: /root/.cache/pip/wheels/20/48/8d/b66d8361a27f58f41ec86640e4fd2640de0403a6367511eab7
Successfully built unidic-lite


In [5]:
#　Reference https://www.pytry3g.com/entry/gensim-word2veReference 
import MeCab
import codecs
import urllib.parse as parser
import urllib.request as request
from bs4 import BeautifulSoup

tagger = MeCab.Tagger("-Owakati")
link = "https://ja.wikipedia.org/wiki/"

#+
#キーワードのリストを受け取り、Wikipediaの該当記事を取得して、fileに出力する。
#-
def kwd2file(kwd,file):
  corpus = []
  for word in keyword:
      with request.urlopen(link + parser.quote_plus(word)) as response:
          html = response.read().decode('utf-8')
          soup = BeautifulSoup(html, "lxml")
          p_tags = soup.find_all('p')
          for p in p_tags:
              corpus.append(tagger.parse(p.text).strip())
  print("corpus size {}".format(len(corpus)))
  print("corpus samples {}".format(corpus[0:5]))
  with codecs.open(file, "w", "utf-8") as f:
      f.write("\n".join(corpus))

keyword = [
           "ロボット","ロケット","コンピュータ","人工知能","自動車","宇宙","機械","道具",
           "人間","目","口","耳","指","手","足","頭","首","声","腹","背","胃","腸",
           "動物","植物","猫","犬","鳩","魚","鳥","昆虫","猿","烏","ミミズ","ナメクジ","蟻","熊","クジラ","イルカ",
           "日本","東京","横浜","アジア","アメリカ","アフリカ","中近東","オセアニア","関東","関西",
           "電気","信号","電波","重力","原子","分子","量子",
           "紙","プラスチック","鉄","アルミ","ビニール","土","火","水",
           "聖徳太子","坂本龍馬","徳川家康","源義経","夏目漱石","宮沢賢治","斎藤茂吉",
           "サッカー","野球","ラグビー","プロレス","テニス","バレーボール","ホッケー","スキー","スケート",
           ]
kwd2file(keyword,'train.txt')
keyword = ["ロボット",] 
kwd2file(keyword,'test.txt') #客観的に性能を示すため、本来、トレーニングデータと全く異なるデータを用いる。
keyword = ["ロケット",]
kwd2file(keyword,'valid.txt') #トレーニングの効果を見るため、本来、トレーニングと別データ。トレーニング用のデータを10等分して、順番に一つを使ったりする。


corpus size 4619
corpus samples ['ロボット （ robot ） は 、 人 の 代わり に 何 等 か の 作業 を 自律 的 に 行う 装置 、 もしくは 機械 の こと 。', '主に 以下 に 大別 する こと が 可能 で ある 。', '近年 で は 無人 機 「 ドローン 」 を 半ば 自律 化 さ せ た もの [ 1 ] も 存在 し 、 自動 運転 車 の 実現 が 視野 に 入っ て き て おり 、 SF の 世界 が 現実 の もの と なり つつ ある 。', '生命 体 に 通常 以上 の 力 を 発揮 さ せる 方策 と し て 何 ら か の 人工 物 を 埋め込ん だり 置き換える など の 方策 を 採っ た 者 は 一般 に 「 サイボーグ 」 など と 呼ば れ 区別 さ れる こと が 多い 。', 'この 言葉 が 初めて 用い られ た の は 、 1920 年 に チェコスロバキア （ 当時 ） の 小説 家 カレル ・ チャペック が 発表 し た 戯曲 『 R . U . R . （ ロッサム 万能 ロボット 商会 ） 』 に おい て で ある が 、 この 作品 で は 現在 認知 さ れ て いる 金属 製 の 機械 で は なく 、 人間 と は 異なる 組成 の 肉体 と 人間 そっくり の 外見 を 持つ もの を 、 化学 的 合成 で 原形 質 を 使っ て 製作 し た もの で 、 現在 の SF で 言う バイオ ノ イド で ある 。']
corpus size 49
corpus samples ['ロボット （ robot ） は 、 人 の 代わり に 何 等 か の 作業 を 自律 的 に 行う 装置 、 もしくは 機械 の こと 。', '主に 以下 に 大別 する こと が 可能 で ある 。', '近年 で は 無人 機 「 ドローン 」 を 半ば 自律 化 さ せ た もの [ 1 ] も 存在 し 、 自動 運転 車 の 実現 が 視野 に 入っ て き て おり 、 SF の 世界 が 現実 の もの と なり つつ ある 。', '生命 体 に 通常 以上 の 力 を 発揮 さ せる 方策 と し て 何 ら か の 人工 物 を 埋め

In [6]:
import os
from io import open
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    #+
    #単語を受け取り、idx2word, word2idxを作成する
    #-
    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        #辞書に単語を登録する
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                #print(words)
                for word in words:
                    self.dictionary.add_word(word)
        #コーパスを単語Indexのリストに変換する
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)
        return ids

#LSTM

In [7]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

#+
#LSTMあるいはGRUを使ったモデル定義
#-
class RNNModel(nn.Module):

    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp) #語彙数x埋め込みサイズの表
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout) #LSTMのオブジェクト
        self.decoder = nn.Linear(nhid, ntoken) #隠れ状態から語彙サイズへの全結合
        self.init_weights()
        self.nhid = nhid #隠れ状態ベクトル次元数
        self.nlayers = nlayers #LSTM/GRUを何枚重ねるか

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        #以下、バッチサイズだけ一挙に計算している
        emb = self.drop(self.encoder(input)) #入力単語の埋め込み表現を取り出して、ドロップアウト
        output, hidden = self.rnn(emb, hidden) #BPTT_SEQ_LENGTH個分の系列を2レイヤーLSTMにかける
        decoded = self.decoder(self.drop(output)) #出力をDropoutかまして語彙サイズへ全結合
        decoded = decoded.view(-1, self.ntoken) #shapeを[[...]]に
        return F.log_softmax(decoded, dim=1), hidden #softmwaxの対数と、新しい隠れ状態ベクトルを返す

    def init_hidden(self, bsz):
        weight = next(self.parameters()) #最初のパラメータ（埋め込み表）から使っている型を取得
        return (weight.new_zeros(self.nlayers, bsz, self.nhid), #同じ型で0初期化したテンソル(2,20,200)を返す
                    weight.new_zeros(self.nlayers, bsz, self.nhid)) #2個目のレイヤ用


In [8]:
# coding: utf-8
import time
import math
import os
import torch
import torch.nn as nn
import easydict

torch.manual_seed(1111) #乱数の初期値設定、結果を安定させる
device = torch.device("cuda" if True else "cpu")
corpus = Corpus('./')
BATCH_SIZE = 20
EVAL_BATCH_SIZE = 10
BPTT_SEQ_LENGTH = 35 #このサイズで逆伝播を停止する。入力が長い場合の対策

def batchify(data, bsz):
    print('batchfy input size {}'.format(data.size()))
    nbatch = data.size(0) // bsz #コーパスの長さをミニバッチサイズで割り、何個のミニバッチかを得る
    data = data.narrow(0, 0, nbatch * bsz) #ミニバッチにハマらなかった部分を捨てる
    data = data.view(bsz, -1).t().contiguous() #データをミニバッチサイズで分割して、行列を入れ替える
    #->shapeが(バッチの個数,バッチサイズ)になる、train_dataなら23154行、20列の行列になる、
    #連続した単語は列方向に並んでいる
    return data.to(device)

#データをミニバッチに分ける
train_data = batchify(corpus.train, BATCH_SIZE)
val_data = batchify(corpus.valid, EVAL_BATCH_SIZE)
test_data = batchify(corpus.test, EVAL_BATCH_SIZE)
#print('train_data.size {}'.format(train_data.size()))

ntokens = len(corpus.dictionary)
model = RNNModel(
    len(corpus.dictionary),
    200,#埋め込みサイズ 
    200,#隠れベクトルサイズ 
    2,#RNNをスタックするレイヤ数, 
    0.2,#ドロップアウト率。複数レイヤの場合、0.2の割合で次段の入力を0にする
    ).to(device)
#print(model)
#print(model.state_dict().keys())
criterion = nn.NLLLoss()

#BPTT_SEQ_LENGTHの区切りで、隠れ状態ベクトルから、計算履歴を消去して、逆伝播させないようにする
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

def get_batch(source, i):
    #iには、0から順番にBPTT_SEQ_LENGTHだけ飛んだ値が入ってくる
    seq_len = min(BPTT_SEQ_LENGTH, len(source) - 1 - i)
    data = source[i:i+seq_len] #列方向からBPTT_SEQ_LENGTHだけ切り出す、連続単語の認識はこの長さの範囲だけで実施
    target = source[i+1:i+1+seq_len].view(-1) #正解単語になるように1個後ろにずらしてBPTT_SEQ_LENGTHだけ切り出す
    return data, target

def train():
    model.train() 
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(BATCH_SIZE) #隠れ状態初期化
    for batch, i in enumerate(range(0, train_data.size(0) - 1, BPTT_SEQ_LENGTH)): #BPTT_SEQ_LENGTH分取り出して繰り返す
        data, targets = get_batch(train_data, i) #shapeが(BPTT_SEQ_LENGTH,BATCH_SIZE)のデータを得る
        #->dataは、35行20列の行列。連続した単語は縦（列方向）に並んでいる。横（行方向）には独立したデータが20個並んでいる
        model.zero_grad() #勾配初期化
        hidden = repackage_hidden(hidden) #隠れ状態の逆伝播を切る
        output, hidden = model(data, hidden) #モデルに通す
        loss = criterion(output, targets) #ロスを求める
        loss.backward() #逆伝播
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25) #勾配の最大が0.25になるように、全勾配に調整かける
        for p in model.parameters(): #学習レート低減
            p.data.add_(-lr, p.grad)
        total_loss += loss.item()

        if batch % 200 == 0 and batch > 0:
            cur_loss = total_loss / 200
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // BPTT_SEQ_LENGTH , lr, elapsed * 1000 / 200, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(data_source):
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(EVAL_BATCH_SIZE) #隠れ状態初期化
    with torch.no_grad(): #評価なので、勾配を計算しない設定にする
        for i in range(0, data_source.size(0) - 1, BPTT_SEQ_LENGTH): #BPTT_SEQ_LENGTH分取り出して繰り返す
            data, targets = get_batch(data_source, i) #shapeが(BPTT_SEQ_LENGTH,BATCH_SIZE)のデータを得る
            output, hidden = model(data, hidden) #モデルに通す
            hidden = repackage_hidden(hidden) #隠れ状態の逆伝播を切る
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(data_source) - 1)

lr = 20
best_val_loss = None
try:
    for epoch in range(1, 10+1): #エポック回数だけ回す 
        epoch_start_time = time.time()
        train() 
        val_loss = evaluate(val_data)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time), val_loss, math.exp(val_loss)))
        if not best_val_loss or val_loss < best_val_loss: #一番良いパラメータを保存する
            with open('jpn_model.pt', 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            lr /= 4.0 #改善しなくなったら、学習レートを下げる
except KeyboardInterrupt:
    print('Exiting from training early')

# Load the best saved model.
with open('jpn_model.pt', 'rb') as f:
    model = torch.load(f)
    model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))

batchfy input size torch.Size([474266])
batchfy input size torch.Size([8038])
batchfy input size torch.Size([4351])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)


| epoch   1 |   200/  677 batches | lr 20.00 | ms/batch 17.23 | loss  7.15 | ppl  1278.24
| epoch   1 |   400/  677 batches | lr 20.00 | ms/batch 16.08 | loss  5.95 | ppl   384.14
| epoch   1 |   600/  677 batches | lr 20.00 | ms/batch 16.10 | loss  5.65 | ppl   283.00
| end of epoch   1 | time: 11.20s | valid loss  5.66 | valid ppl   287.57
| epoch   2 |   200/  677 batches | lr 20.00 | ms/batch 16.48 | loss  5.34 | ppl   208.81
| epoch   2 |   400/  677 batches | lr 20.00 | ms/batch 16.19 | loss  5.17 | ppl   175.33
| epoch   2 |   600/  677 batches | lr 20.00 | ms/batch 16.25 | loss  5.12 | ppl   167.45
| end of epoch   2 | time: 11.15s | valid loss  5.29 | valid ppl   198.37
| epoch   3 |   200/  677 batches | lr 20.00 | ms/batch 16.48 | loss  4.94 | ppl   140.31
| epoch   3 |   400/  677 batches | lr 20.00 | ms/batch 16.46 | loss  4.85 | ppl   128.08
| epoch   3 |   600/  677 batches | lr 20.00 | ms/batch 16.52 | loss  4.84 | ppl   125.84
| end of epoch   3 | time: 11.25s | valid 

In [9]:
import torch
torch.manual_seed(1111)
device = torch.device("cuda" if True else "cpu")
with open('./jpn_model.pt', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()
corpus = Corpus('./')
ntokens = len(corpus.dictionary)
hidden = model.init_hidden(1) #バッチサイズを１にして初期化
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device) #先頭単語をランダムに選ぶ
with torch.no_grad():  
    sentence = '' #生成文
    for i in range(500):
        output, hidden = model(input, hidden) #outputは語彙サイズの確率ベクトル[[...]]
        word_weights = output.squeeze().div(1.0).exp().cpu() #語彙サイズのEXP(x)
        word_idx = torch.multinomial(word_weights, 1)[0] #確率が一番大きな単語のIndexをとる
        input.fill_(word_idx) #この予測単語を、次のループ時のinputに設定
        word = corpus.dictionary.idx2word[word_idx]
        sentence += (word  + ('\n' if i % 20 == 19 else ' '))
    print(sentence)

年 に は 477 、 援 染色 思い 放棄 の ゲーム ・ 茂吉 を 使用 し た 大会 で は
ネコ 圏 で は なく 、 航路 は 、 下腹 と 呼ば れ 、 喪 あるいは 眼球 を まとめ た
1945 年 の メス が 出馬 縫い合わさ が 、 内部 の 超 一 例 で ある 。 第 二 次
世界 大戦 後 も 5 億 ドル の ピアニスト 話者 を 発する 。 また 、 近世 は G 2 万
5 千 万 台 と さ れ 、 大 世界 の 安定 な かたち で は 40 と さ れる
世界 の 約 100 % （ また 9 . dmbox m ) で あっ た [ 105 ]。 <eos> また
、 制圧 や ともない の なか で は （ 通常 と は 目撃 さ れる ） が 、 タマムシ の
汚職 オキアミ など 第 1 在学 に わたっ て は 関係 の ない こと で 、 生活 を する 三
人 は 必ず しも 大きく の 横向き で ある 。 両者 の 中 で 食べ られ 、 それ まで の
とおり 、 隔絶 し た 構造 を よく 使用 する など は 口 や 動物 の 他 の 哺乳 類
が 著しく 連続 的 な 特徴 で ある [ 107 ]。 <eos> ジュラ 紀 に は 軟体 種 に は
鰓 ごみ が 上下 型 硫黄 を 集会 し て 最大 の 歯 を 制限 し て いる が 、
幾 つ か の 生態 系 を 持つ 例 で は 判断 個体 が 市内 行わ れる が 、 アルファ
者 の 中 に は 森林 原料 の 瞳孔 を D と 構成 する 例 も あっ た が 、
なか に も ツバメ を 溜め て うち 度合い を 知る べき と し て 行わ れる 。 変異 じぎ
を 攻撃 し て いる 鳥類 も 、 実在 し て いる 。 出産 中 より 1 マルク を 上行
し て 消費 する こと で 腓 用 型 の 結膜 に ある ため の オットー 小型 231 . 5
割 の 窒素 を 掌握 し て いる が 、 眼杯 の 検査 の ため の 摂取 が 認め られ
たり 、 人間 を 孵化 する 者 も いる 。 <eos> 個眼 は 1 境界 6 色 ）[ 11 ]。
1995 年 に は 網膜 が 変更 さ れ て い た が 